# Imports

In [2]:
import os
import sys
import json
import xarray as xr
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

In [3]:
# go up until we are in the project base directory
base_dir = os.getcwd()
while base_dir.split('/')[-1] != 'provide':
    base_dir = os.path.normpath(os.path.join(base_dir, '..'))

# add paths for tools and data
things_to_add = ['general_tools', 'aggregation_tools', 'general_data_for_aggregation']
for thing in things_to_add:
    sys.path.append(os.path.join(base_dir, thing))

# import stuff we need
from general_tools import check_if_notebook, mkdir
from oggm_result_filepath_and_realisations import (gcms_mesmer, quantiles_mesmer,
    scenarios_mesmer, oggm_result_dir, provide_regions, raw_oggm_output_file)
from aggregation_preprocessing import open_grid_from_dict
from actual_aggregation import open_files_and_aggregate_on_map, aggregating_scenario, check_slurm_done

In [4]:
# Use this to conditionally execute tests/debugging
if check_if_notebook():
    is_notebook = True
else:
    is_notebook = False

# Define directories

In [4]:
resolution_dir = 'total_data'

In [5]:
preprocess_region_dict_outpath = os.path.join(base_dir, 'basins', resolution_dir)
mkdir(preprocess_region_dict_outpath);

# Open data

In [6]:
with open(os.path.join(preprocess_region_dict_outpath, "preprocessed_region_grids.json"), 'r') as f:
    region_structure_dict = json.load(f)

# Code for running on cluster

## define experiments for multiprocessing

In [7]:
# create dict for sbatch --array=
slurm_arrays = {}

count = 1
for region in region_structure_dict.keys():
    for scenario in scenarios_mesmer:
        slurm_arrays[str(count)] = (region, scenario)
        count += 1

if is_notebook:
    print(slurm_arrays)

{'1': ('indigirka', 'CurPol'), '2': ('indigirka', 'GS'), '3': ('indigirka', 'LD'), '4': ('indigirka', 'ModAct'), '5': ('indigirka', 'Ref'), '6': ('indigirka', 'Ren'), '7': ('indigirka', 'Neg'), '8': ('indigirka', 'SP'), '9': ('indigirka', 'ssp119'), '10': ('indigirka', 'ssp534-over'), '11': ('ob', 'CurPol'), '12': ('ob', 'GS'), '13': ('ob', 'LD'), '14': ('ob', 'ModAct'), '15': ('ob', 'Ref'), '16': ('ob', 'Ren'), '17': ('ob', 'Neg'), '18': ('ob', 'SP'), '19': ('ob', 'ssp119'), '20': ('ob', 'ssp534-over'), '21': ('brahmaputra', 'CurPol'), '22': ('brahmaputra', 'GS'), '23': ('brahmaputra', 'LD'), '24': ('brahmaputra', 'ModAct'), '25': ('brahmaputra', 'Ref'), '26': ('brahmaputra', 'Ren'), '27': ('brahmaputra', 'Neg'), '28': ('brahmaputra', 'SP'), '29': ('brahmaputra', 'ssp119'), '30': ('brahmaputra', 'ssp534-over'), '31': ('ganges', 'CurPol'), '32': ('ganges', 'GS'), '33': ('ganges', 'LD'), '34': ('ganges', 'ModAct'), '35': ('ganges', 'Ref'), '36': ('ganges', 'Ren'), '37': ('ganges', 'Neg'

## run current experiment

In [ ]:
if not is_notebook:
    slurm_id = os.environ.get('ARRAY_ID', None)
    # convert slurm array to region and scenario
    region, scenario = slurm_arrays[slurm_id]

    # save results on cluster and copy at the end in run_slurm-file
    working_dir_cluster = os.environ.get('OGGM_WORKDIR', None)

    aggregated_data_outpath = os.path.join(
        working_dir_cluster, 'aggregated_data')
    mkdir(aggregated_data_outpath);

    aggregated_data_intermediate_outpath = os.path.join(
        preprocess_region_dict_outpath,
        'aggregated_data_intermediate')
    mkdir(aggregated_data_intermediate_outpath);

    aggregating_scenario(
            target_name=region,
            target_structure_dict=region_structure_dict,
            scenario=scenario,
            output_folder=aggregated_data_outpath,
            oggm_result_dir=oggm_result_dir,
            raw_oggm_output_file=raw_oggm_output_file,
            intermediate_data_folder=aggregated_data_intermediate_outpath,
            variables=['volume', 'area', 'thinning_rate', 'runoff'],
            risk_variables=['volume', 'area'],
            risk_thresholds=np.append(np.arange(10, 91, 10), [99]),  # in % melted of 2020, 10% means 10% of 2020 melted
            time_steps=np.arange(2015, 2101, 5),
            reset_files=False
        )

## check which experiments failed for rerunning

In [5]:
if is_notebook:
    check_slurm_done(475437)

Files without 'SLURM DONE':
sbatch --array=587,699,523,252,338,336,456,748,352,598,416,360,182,625,609,330,433,600,716,228,406,302,305,136,656,313,624,385,506,574,169,321,279,618,664,317,18,151,736,579,273,322,667,206,116,399,100,720,341,373,636,250,692,582,389,117,483,380,272,10,363,412,460,232,653,125,685,122,583,300,73,197,714,403,256,218,216 run_slurm_aggregation_workflow.sh


In [6]:
if is_notebook:
    check_slurm_done(476465)

Files without 'SLURM DONE':
sbatch --array=117,363,232,116,10,433,699 run_slurm_aggregation_workflow.sh


In [7]:
if is_notebook:
    check_slurm_done(476556)

Files without 'SLURM DONE':
sbatch --array=232,433 run_slurm_aggregation_workflow.sh


In [8]:
if is_notebook:
    check_slurm_done(476580)

All files contain 'SLURM DONE'.
